In [1]:
import cv2
import numpy as np
import pyzbar.pyzbar as pyzbar
from pyzbar.pyzbar import ZBarSymbol
from time import sleep
from spider import Spider
from math import atan2

# have to edit this file, remove convex hull code to get orientation
print(pyzbar.__file__)

/Users/joncomo/miniconda2/envs/py36/lib/python3.6/site-packages/pyzbar/pyzbar.py


In [2]:
s = Spider()

trying to connect to:  raspberrypi.local 5005
connected, use send_recv(msg) to send data


In [3]:
accel = s.move(np.random.randn(8) * .2, msg_type=1, sleep_time=.1, accel=30)
print(accel)
sleep(1)

[0.16, 0.02, 9.98]


In [4]:
cap = cv2.VideoCapture(1)

In [14]:
def track(img):
    decodedObjects = []
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (0,0), fx=0.6, fy=0.6)
    decodedObjects = pyzbar.decode(img, symbols=[ZBarSymbol.QRCODE])
    
    # Loop over all decoded objects
    points = []
    
    for decodedObject in decodedObjects: 
        points = decodedObject.polygon
 
    # If the points do not form a quad, find convex hull
    hull = points;
     
    # Number of points in the convex hull
    n = len(hull)
    
    if n == 0:
        return img, -1, -1, -1
 
    # Draw the convext hull
    for j in range(0,n):
        cv2.line(img, hull[j], hull[ (j+1) % n], (255,0,0), 3)
        
    # center
    cx = 0
    cy = 0
    
    # angle
    angle = np.arctan2(hull[0][1] - hull[1][1], hull[0][0] - hull[1][0])
    angle -= np.pi/2
    
    for i in range(n):
        cx += hull[i][0]
        cy += hull[i][1]
        
    cx /= float(n)
    cy /= float(n)
    
    cv2.circle(img, (int(cx), int(cy)), 5, color=(255,255,255), thickness=2)
    cv2.circle(img, (int(cx + np.cos(angle) * 50), int(cy + np.sin(angle) * 50)), 5, color=(255,255,255), thickness=2)
    
    cv2.putText(img,'{}, {}'.format(cx, cy), (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    return img, cx, cy, angle

while True:
    ret, img = cap.read()
    img, cx, cy, angle = track(img)
    
    accel = s.move(np.zeros(8), msg_type=1, sleep_time=.01, accel=30)
    x = accel[0]
    y = -accel[1]
    a2 = atan2(y, x) + np.pi/2.0
    cx = cx + 50 * np.cos(angle + a2)
    cy = cy + 50 * np.sin(angle + a2)
    
    img = cv2.circle(img, (int(cx+x), int(cy+y)), 5, color=(255,0,0), thickness=2)
    
    # Display results 
    cv2.imshow("Results", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break